高阶pandas

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_note_interactivity = "all"
%matplotlib inline


import os
import sys
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import matplotlib.pyplot as plt
import seaborn as sns

1. 分类数据

——————背景及目标

——————pandas中的Categorical类型

——————使用Categorical对象进行计算

————————————————使用分类获得更高性能

——————分类方法

————————————————创建拥有建模的虚拟变量

2. 高阶GroupBy应用

——————分组转换和“展开”GroupBy

——————分组的时间重新采样

3. 方法链技术

——————pipe方法


1. 分类数据

pandas中的Categorical类型。之前在分箱中出现过。

1.1 背景和目标

In [3]:
# 一个列经常会包含重复值，这些重复值是一个小型的不同值的集合。
# 之前用到了unique和value_counts函数，允许用户从一个数组中提取不同的值并分别计算这些不同值的频率：

values = pd.Series(['apple', 'orange', 'apple', 'apple'] * 2)
values

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
dtype: object

In [4]:
pd.unique(values)

array(['apple', 'orange'], dtype=object)

In [5]:
pd.value_counts(values)

apple     6
orange    2
dtype: int64

In [6]:
values.value_counts()

apple     6
orange    2
dtype: int64

In [7]:
# 维度表(主要观测值的整数键作为索引存储在维度表中)数据索引之表

values = pd.Series([0, 1, 0, 0] * 2)
dim = pd.Series(['apple', 'orange'])
values

0    0
1    1
2    0
3    0
4    0
5    1
6    0
7    0
dtype: int64

In [9]:
dim

0     apple
1    orange
dtype: object

In [10]:
dim.take(values)

0     apple
1    orange
0     apple
0     apple
0     apple
1    orange
0     apple
0     apple
dtype: object

* 重命名类别

* 在不改变类别顺序的情况下添加一个新的列

In [11]:
fruits = ['apple', 'orange', 'apple', 'apple'] * 2

In [12]:
N = len(fruits)

In [13]:
N

8

In [14]:
df = pd.DataFrame({'fruit': fruits,
                   'basket_id': np.arange(N),
                   'count': np.random.randint(3, 15, size=N),
                   'weight': np.random.uniform(0, 4, size=N)},
                  columns=['basket_id', 'fruit', 'count', 'weight'])

In [15]:
df

,basket_id,fruit,count,weight
0,0,apple,12,3.503329
1,1,orange,14,3.323108
2,2,apple,6,2.723947
3,3,apple,3,0.508614
4,4,apple,8,3.345693
5,5,orange,11,1.568063
6,6,apple,5,0.238171
7,7,apple,8,1.965434


In [16]:
# df['fruit']是一个Python字符串对象组成的数组，通过调用函数将其转换为Categorical对象。

fruit_cat = df['fruit'].astype('category')
fruit_cat

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): [apple, orange]

In [18]:
# fruit_cate的值并不是Numpy数组，而是pandas.Categorical的实例

fruit_cat

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): [apple, orange]

In [19]:
c = fruit_cat.values
type(c)

pandas.core.categorical.Categorical

In [20]:
c.categories

Index(['apple', 'orange'], dtype='object')

In [21]:
# 通过分配已转换的结果将DataFrame的一列转换为Categorical对象：

df['fruit'] = df['fruit'].astype('category')
df.fruit

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): [apple, orange]

In [23]:
# 同样可以通过其他Python序列来生成pandas.Categorical实例

my_cate = pd.Categorical(['foo', 'bar', 'baz', 'foo', 'bar'])
my_cate

[foo, bar, baz, foo, bar]
Categories (3, object): [bar, baz, foo]

In [24]:
# 如果已经从另一个数据源获得了分类编码数据，可以通过from_codes构造函数：

categories = ['foo', 'bar', 'baz']

codes = [0, 1, 2, 0, 0, 1]
my_cates2 = pd.Categorical.from_codes(codes, categories=categories)
my_cates2

[foo, bar, baz, foo, foo, bar]
Categories (3, object): [foo, bar, baz]

In [25]:
#默认是不排序的，需要手动指定排序

ordered_cat = pd.Categorical.from_codes(codes, categories, ordered=True)
ordered_cat

[foo, bar, baz, foo, foo, bar]
Categories (3, object): [foo < bar < baz]

1.3 使用Categorical对象进行计算

In [26]:
# pandas中的groupby函数在于Categorical对象协同工作时性能更好。

In [27]:
# 使用pandas.qcut进行分箱

np.random.seed(12345)
draws = np.random.randn(1000)
draws[:5]

array([-0.20470766,  0.47894334, -0.51943872, -0.5557303 ,  1.96578057])

In [28]:
bins = pd.qcut(draws, 4)

In [33]:
bins

[(-0.684, -0.0101], (-0.0101, 0.63], (-0.684, -0.0101], (-0.684, -0.0101], (0.63, 3.928], ..., (-0.0101, 0.63], (-0.684, -0.0101], [-2.949, -0.684], (-0.0101, 0.63], (0.63, 3.928]]
Length: 1000
Categories (4, object): [[-2.949, -0.684] < (-0.684, -0.0101] < (-0.0101, 0.63] < (0.63, 3.928]]

In [34]:
# 利用qcut函数中的labels参数来指定箱名

bins = pd.qcut(draws, 4, labels = ['Q1', 'Q2', 'Q3', 'Q4'])
bins

[Q2, Q3, Q2, Q2, Q4, ..., Q3, Q2, Q1, Q3, Q4]
Length: 1000
Categories (4, object): [Q1 < Q2 < Q3 < Q4]

In [35]:
bins.codes[:10]

array([1, 2, 1, 1, 3, 3, 2, 2, 3, 3], dtype=int8)

In [36]:
# 被标记的bins分类数据并不包括数据中箱体片接的相关信息，因此可以使用groupby来提取一些汇总统计值
# 这段儿代码恐怕以后会经常用到，用来做直方图什么的

bins = pd.Series(bins, name='quartile')
results = (pd.Series(draws)
          .groupby(bins)
          .agg(['count', 'min', 'max'])
          .reset_index())
results

,quartile,count,min,max
0,Q1,250,-2.949343,-0.685484
1,Q2,250,-0.683066,-0.010115
2,Q3,250,-0.010032,0.628894
3,Q4,250,0.634238,3.927528


In [37]:
results['quartile']

0    Q1
1    Q2
2    Q3
3    Q4
Name: quartile, dtype: category
Categories (4, object): [Q1 < Q2 < Q3 < Q4]

1.3.1 使用分类来获得更高的性能

如果用户在特定的数据集上做了大量分析，将数据转换为分类数据可以产生大幅的性能提升。

df中一列的分类版本通常也会明显使用更少内存。

In [49]:
N = 10000000
draws = pd.Series(np.random.randn(N))
labels = pd.Series(['foo', 'bar', 'barz', 'qux']) * (N // 4)

In [50]:
categories = labels.astype('category')

In [51]:
labels.memory_usage()

112

In [52]:
categories.memory_usage()

116

In [53]:
%time _ = labels.astype('category')

Wall time: 501 µs


1.4 分类方法

Series包含的分类数据拥有一些特殊方法，这些方法类似于Series.str的特殊字符串方法。这些方法提供了更快捷访问类别和代码的方式。

In [54]:
s = pd.Series(['a', 'b', 'c', 'd'] * 2)

In [55]:
cat_s = s.astype('category')

In [56]:
cat_s

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (4, object): [a, b, c, d]

In [57]:
# 特殊属性cat提供了对分类方法的访问：

cat_s.cat.codes

0    0
1    1
2    2
3    3
4    0
5    1
6    2
7    3
dtype: int8

In [58]:
cat_s.cat.categories

Index(['a', 'b', 'c', 'd'], dtype='object')

In [59]:
# 如果发现该数据的实际类别集合超过了数据中观察到的四个值。可以使用set_categories方法来改变类别。

In [60]:
actual_categories = ['a', 'b', 'c', 'd', 'e']
cat_s2 = cat_s.cat.set_categories(actual_categories)
cat_s2

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (5, object): [a, b, c, d, e]

In [61]:
cat_s.value_counts()

d    2
c    2
b    2
a    2
dtype: int64

In [63]:
# 按照指定的分类序列进行计数
cat_s2.value_counts()

d    2
c    2
b    2
a    2
e    0
dtype: int64

In [81]:
cat_s3 = cat_s[cat_s.isin(['a', 'b'])]
cat_s3

0    a
1    b
4    a
5    b
dtype: category
Categories (4, object): [a, b, c, d]

In [82]:
cat_s3.cat.remove_unused_categories()

0    a
1    b
4    a
5    b
dtype: category
Categories (2, object): [a, b]

pandas中Series对象的分类方法：

* add_categories:  将新的类别添加到已有类别的尾部

* as_ordered:  对类别排序

* as_unordered:  使类别无序

* remove_categories:  去除类别，将被移除的值设置为null

* remove_unused_categories:  去除所有没有出现在数据中的类别

* rename_categories:  使用新的类别名里替代现有的类别，不会改变类别的数量

* reroder_categories:  与rename_categories类似，但结果是经过排序的类别

* set_categories:  用指定一组新类别替换现有类别，可以添加或删除类别

1.4.1 创建用于建模的虚拟变量

In [64]:
# 在使用统计数据或机器学习工具时，通常会将分类数据转换为虚拟变量，也称为one-hot编码。该过程会产生一个DF，每个不同的类别都是它的一列。

In [65]:
cat_s = pd.Series(['a', 'b', 'c', 'd'] * 2, dtype='category')

In [66]:
pd.get_dummies(cat_s)

,a,b,c,d
0,1,0,0,0
1,0,1,0,0
2,0,0,1,0
3,0,0,0,1
4,1,0,0,0
5,0,1,0,0
6,0,0,1,0
7,0,0,0,1


In [68]:
df = pd.DataFrame([{'name': 'zyl', 
                   'age': 18,
                   'sex': 'Male',
                   'math': 100},
                   {'name': 'lsl', 
                   'age': 8,
                   'sex': 'Female',
                   'math': 0},
                   {'name': 'tl', 
                   'age': 88,
                   'sex': 'Male',
                   'math': 70}])

In [71]:
df

,age,math,name,sex
0,18,100,zyl,Male
1,8,0,lsl,Female
2,88,70,tl,Male


In [73]:
pd.get_dummies(df.sex)

,Female,Male
0,0,1
1,1,0
2,0,1


2. 高阶GroupBy应用

在groupby中的拆分-应用-联合模式中，除了apply函数之外还有一个内建方法transform

它与apply方法比较类似，但会给用户使用的函数种类加上更多的限制：

* transform可以产生一个标量值，并广播到个分组的尺寸数据中；

* 可以产生一个与输入分组尺寸相同的对象；

* 不可改变它的输入。

In [76]:
df = pd.DataFrame({'key': ['a', 'b', 'c'] * 4, 'value': np.arange(12.)})
df

,key,value
0,a,0.0
1,b,1.0
2,c,2.0
3,a,3.0
4,b,4.0
5,c,5.0
6,a,6.0
7,b,7.0
8,c,8.0
9,a,9.0


In [79]:
g = df.groupby('key').value
print(g)
g.mean()

key
a    4.5
b    5.5
c    6.5
Name: value, dtype: float64

In [78]:
#假设需要产生一个Series，它的尺寸和df['value']一样，但值都被按'key'分组的均值替代。可以向transform传递匿名函数lambda x: x.mean()

g.transform(lambda x:x.mean())

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [80]:
#对于内建聚合函数，可以向groupby中的agg方法一样传递一个字符串别名：

g.transform('mean')

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [83]:
#与apply方法类似，transform可以与返回Series的函数一起使用，但结果必须与输入有相同的大小。

In [84]:
g.transform(lambda x: x.rank(ascending=False))

0     4.0
1     4.0
2     4.0
3     3.0
4     3.0
5     3.0
6     2.0
7     2.0
8     2.0
9     1.0
10    1.0
11    1.0
Name: value, dtype: float64

In [85]:
def normalize(x):
    return (x - x.mean()) / x.std()

In [86]:
g.transform(normalize)

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

In [87]:
g.apply(normalize)

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

In [89]:
%time g.transform('mean')

Wall time: 1.96 ms


0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [91]:
%time g.apply(normalize) #看来还是transfrom与groupby对象的相性更好，速度快不少，但是约束也更多

Wall time: 13 ms


0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

In [92]:
normalized = (df['value'] - g.transform('mean') / g.transform('std'))

In [93]:
normalized

0    -1.161895
1    -0.420094
2     0.321707
3     1.838105
4     2.579906
5     3.321707
6     4.838105
7     5.579906
8     6.321707
9     7.838105
10    8.579906
11    9.321707
Name: value, dtype: float64

2.2 分组的时间采样

In [94]:
N = 15

In [95]:
times = pd.date_range('2017.5.20 00:00', freq='1min', periods=N)
df = pd.DataFrame({'time': times,
                   'value': np.arange(N)})

In [96]:
df

,time,value
0,2017-05-20 00:00:00,0
1,2017-05-20 00:01:00,1
2,2017-05-20 00:02:00,2
3,2017-05-20 00:03:00,3
4,2017-05-20 00:04:00,4
5,2017-05-20 00:05:00,5
6,2017-05-20 00:06:00,6
7,2017-05-20 00:07:00,7
8,2017-05-20 00:08:00,8
9,2017-05-20 00:09:00,9


In [97]:
df.set_index('time').resample('5min').count()

,value
time,
2017-05-20 00:00:00,5
2017-05-20 00:05:00,5
2017-05-20 00:10:00,5


In [98]:
df2 = pd.DataFrame({'time': times.repeat(3),
                    'key': np.tile(('a', 'b', 'c'), N),
                    'value': np.arange(N * 3.)})

In [99]:
df2[:7]

,key,time,value
0,a,2017-05-20 00:00:00,0.0
1,b,2017-05-20 00:00:00,1.0
2,c,2017-05-20 00:00:00,2.0
3,a,2017-05-20 00:01:00,3.0
4,b,2017-05-20 00:01:00,4.0
5,c,2017-05-20 00:01:00,5.0
6,a,2017-05-20 00:02:00,6.0


In [100]:
#为每个key进行重采样

time_key = pd.TimeGrouper('5min')

In [101]:
resampled = (df2.set_index('time').groupby(['key', time_key]).sum())

In [102]:
resampled

value
key time                      
a   2017-05-20 00:00:00   30.0
    2017-05-20 00:05:00  105.0
    2017-05-20 00:10:00  180.0
b   2017-05-20 00:00:00   35.0
    2017-05-20 00:05:00  110.0
    2017-05-20 00:10:00  185.0
c   2017-05-20 00:00:00   40.0
    2017-05-20 00:05:00  115.0
    2017-05-20 00:10:00  190.0

In [103]:
resampled.reset_index()

,key,time,value
0,a,2017-05-20 00:00:00,30.0
1,a,2017-05-20 00:05:00,105.0
2,a,2017-05-20 00:10:00,180.0
3,b,2017-05-20 00:00:00,35.0
4,b,2017-05-20 00:05:00,110.0
5,b,2017-05-20 00:10:00,185.0
6,c,2017-05-20 00:00:00,40.0
7,c,2017-05-20 00:05:00,115.0
8,c,2017-05-20 00:10:00,190.0


* 需要注意的是pandas.TimeGrouper的一个限制是时间必须是Series或者DF的索引。

3. 方法链技术

在向数据集应用一系列变换时，可能会在过程中创建许多临时变量，而这些变量在分析中从未使用过。

In [ ]:
'''比如
df = load_data()
df2 =df[df['col2'] < 0]
df2['col1_demeaned'] = df2['col1'] - df2['col1'].mean()
result = df2.groupby('key').col1_demeaned.std()
'''

常见的非函数方式

In [106]:
'''
df2 = df.copy()
df2['k'] = v
'''

"\ndf2 = df.copy()\ndf2['k'] = v\n"

函数赋值方法

In [109]:
'''
result = (df2.assign(col1_demeaned=df2.col1 - df2.col2.mean())
          .groupby('keay')
          .col1_demeaned.std())
'''

"\nresult = (df2.assign(col1_demeaned=df2.col1 - df2.col2.mean())\n          .groupby('keay')\n          .col1_demeaned.std())\n"

In [ ]:
'''
df = load_data()
df2 = df[df['col2'] < 0]
'''
#等价于
'''
df = (load_data()[lambda x: x['col2'] < 0])

这里load_data方法的结果没有复制给一个变量，因此传递进[]的函数将会被绑定到方法链那一阶段的对象上
'''

In [110]:
'''
因此可以最终写成：

result = (load_data()[lambda x: x.col2 < 0]
          .assign(col1_demeaned=lambda x: x.col1 - x.col1.mean())
          groupbu('key')
          .col1_demeaned.std())
'''

"\n因此可以最终写成：\n\nresult = (load_data()[lambda x: x.col2 < 0]\n          .assign(col1_demeaned=lambda x: x.col1 - x.col1.mean())\n          groupbu('key')\n          .col1_demeaned.std())\n"

3.1 pipe方法

使用内建的pandas函数和我们刚才看到的用可调参数进行方法链接的方式，用户可以完成多种工作。然而，优势用户需要使用自定义的函数或来自第三方库的函数。这就是pipe出现的原因。

In [111]:
'''
a = f(df. arg1=v1)
b = g(a, v2, arg3=v3)
c = h(b, arg4=v4)
'''


'\na = f(df. arg1=v1)\nb = g(a, v2, arg3=v3)\nc = h(b, arg4=v4)\n'

In [ ]:
'''
利用pipe重写：

result = (df.pipe(f, arg1=v1)
          .pipe(g, v2, arg3=v3)
          .pipe())
'''

表达式f(df)与df.pipe(f)是等价的，但是pipe令链式调用更加方便。

In [112]:
'''
g = df.groupby(['key1', 'key2'])
df['col1'] = df['col1'] - g.transform('mean')
'''

"\ng = df.groupby(['key1', 'key2'])\ndf['col1'] = df['col1'] - g.transform('mean')\n"

In [113]:
'''
对多个列去除均值并方便地改变分组键。

def group_demean(df, by, cols):
    result = df.copy()
    g = df.groupby(by)
    for c in cols:
        result[c] = df[c] - g[c].transform('mean')
    return result
'''

"\ndef group_demean(df, by, cols):\n    result = df.copy()\n    g = df.groupby(by)\n    for c in cols:\n        result[c] = df[c] - g[c].transform('mean')\n    return result\n"

In [114]:
'''
result = (df[df.col1 < 0]
         .pipe(group_demean, ['key1', 'key2'], ['col1']))
'''

"\nresult = (df[df.col1 < 0]\n         .pipe(group_demean, ['key1', 'key2'], ['col1']))\n"